# Pandas vs SQL
Earlier in the semester, we examined how we can wrangle tables stored in a database with SQL. Here we see how Python can accomplish the same tasks, but from within the powerful Python environmnent and in a reproduceable fashion. 

In this exercise we revisit the analysis of the Seamap400 data of marine species observations - the same dataset we used in Exercise 2.3. While we are not covering this in much depth at all, these exercises should serve as a good introduction to what Pandas can do and how rows and columns are referenced in DataFrames.  

In [ ]:
#Import the modules, we'll need Numpy as well for its datatypes
import pandas as pd
import numpy as np

## Importing the csv and checking data types

In [ ]:
#Import the Seamap400.csv file to a DataFrame
dfSeamap = pd.read_csv('../Data/Seamap400.csv', index_col = 'ID')
dfSeamap.head(2)

In [ ]:
#Check the data types of the import
dfSeamap.dtypes

In [ ]:
#Convert the obs_date from generic 'object' data type to a datetime object
dfSeamap['obs_date'] = pd.to_datetime(dfSeamap['obs_date'])
print "Earliest = ", dfSeamap['obs_date'].min()
print "Latest = ", dfSeamap['obs_date'].max()

## Exploring our dataFrame

In [ ]:
dfSeamap.sp_common.unique()

### Selecting columns from a dataFrame
Selecting columns is analogous to the SQL `select ... from` clause, but the syntax in Pandas is a bit different.

#### Selecting a single column
Selecting a single column is easy as specifying the column name, either between brackets with the field name in quotes, or using the dot notation. (*The former is preferred, in case your field name is the the same as another property or method of a dataFrame.*)

When only one column is selected, the object returned is actually not a  dataFrame, but a pandas **series** object, which is quite similar to a one-dimensional NumPy array. However, the Pandas series is limited to one dimension...

In [ ]:
#Select just the sp_common field into a new view
dfSelect = dfSeamap['sp_common']
dfSelect.head()

In [ ]:
#This also works
commonNames = dfSeamap.sp_common
type(commonNames)

In [ ]:
#The object returned is a series and we can get records from its index
commonNames[:5] #Returns the first 5 records

In [ ]:
#What code would you use to get the last 5 records? (Replace the ???)
commonNames[-5:]

#### Selecting multiple columns
Selecting multiple columns can be done by including a list of the column names. *Note the double set of brackets: the outer set is the syntax of the dataFrame object, and the inner set denotes the list of field names we want.*

Here, the object returned resembles a dataFrame, not a series (since Pandas' series can only be one-dimension). However, in actuality, this is simply a **view** of the original dataFrame, not a new object - much like query views in SQL...

In [ ]:
dfSelect = dfSeamap[['sp_common','sp_class']]
type(dfSelect)

### Selecting rows in a dataFrame
Selecting rows, analogous to the `select...where` statement in SQL, is done by creating boolean masks of the criteria you want and then applying those masks. This can be done explicitily as two steps, or more commonly in a single, compount step. 

Before making our selection, let's examine how we can easily extract a list of valid options using the `unique` property.

In [ ]:
#List the unique values in the 'sp_class' field.
dfSeamap['sp_class'].unique()

#### Selecting rows in two steps:

In [ ]:
#Step 1: Create the boolean mask
theMask = dfSeamap['sp_class'] == 'Mammalia'
theMask[2150:2156]

In [ ]:
#Step 2: Applying the mask to return only 'true' records
dfMammals = dfSeamap[theMask]
dfMammals.sample(4)

#### Selecting rows in one step:

In [ ]:
#Get all the mammals and report all unique scientific names
dfMammals = dfSeamap[dfSeamap['sp_class'] == 'Mammalia']
dfMammals['scientific'].unique()

#### Complex row selections

In [ ]:
dfSeamap['obs_date'].max()# > startDate).sum()

In [ ]:
#Select mammal records in the 2nd half of 2006
mammalMask = dfSeamap['sp_class'] == 'Mammalia'

#Create date objects for the start and end dates
startDate = np.datetime64('2006-07-01')
endDate = np.datetime64('2007-01-01')

#Create the date masks
startMask = (dfSeamap['obs_date'] >= startDate) 
endMask = (dfSeamap['obs_date'] < endDate)

#Apply the masks, using the bitwise '&' to return rows where all masks are true
dfSelect2 = dfSeamap[mammalMask & startMask & endMask]
dfSelect2['scientific'].unique()

#### Return rows matching a substring
The `.str` function on a column allows us to use some string operations  on the values in that field.  Here we use the string `startswith` function to return all rows where the row's value starts with 'Delphin'. See https://pandas.pydata.org/pandas-docs/stable/text.html for other string operations.

In [ ]:
#Select rows where the scientific name starts with "Delphin" 
dolphinMask = dfSeamap['scientific'].str.startswith('Delphin')
dfDolphins = dfSeamap[dolphinMask]

#Use the nunique function to just return the number 
#  of unique scientific names
dfDolphins['scientific'].nunique()

#### More complex queries with `apply` and  `lambda`
For maximum flexibility, we can actually write our own functions to be applied to each value in a column (or multiple columns). This is done using the `apply` function to a dataFrame and then specifying the subcode we want to use with Python's `lambda` statement. (This seems fairly complex at first, but it actually somewhat straightforward -- and can be very useful...)

In [ ]:
#Create a mask by searching each row for the string "Whale" and apply the mask
# to list the scientific names of these records
whaleMask = dfSeamap['sp_common'].apply(lambda x: 'Whale' in x)
dfWhale = dfSeamap[whaleMask]
dfWhale['scientific'].unique()

## Grouping/Aggregating data
Pandas can aggregate data on values like SQL as well. We do this with the `groupby` statement

In [ ]:
#Count the observations by common name
grpSpCommon = dfSeamap.groupby('sp_common')
grpSpCommon['sp_common'].count()

We can also show *all* the summary stats with the dataFrame's `describe` function.

In [ ]:
grpSpCommon.describe()

## Joining tables
Also like SQL, pandas can join tables. Below we'll create two tables from our aggregated data: one will list the minimum of the latitude and longitude columns, and the second will list the maximum values. Then we'll join these two tables and compute the geographic extent of each species observations. 

In [ ]:
#Create dataFrames of the minimum and then maximum of the lat and lng fields
minCoords = grpSpCommon['latitude','longitude'].min()
maxCoord = grpSpCommon['latitude','longitude'].max()

In [ ]:
# Have a look at what is returned (for the min table)
minCoords.head()

In [ ]:
#Use the Pandas 'merge' command to join the two tables
sppExtent = pd.merge(left=minCoords,     #Specifies the left table
                     right=maxCoord,     #Specifies the right table
                     how = 'inner',      #Specifies the type of join
                     left_index=True,    #Use the index of the left table as the join item
                     right_index=True)   #Use the index of the right table as the join item
#Have a look
sppExtent.head()

In [ ]:
#Rename the columns to the values in the list provided
sppExtent.columns = ['minX','minY','maxX','maxY']
sppExtent.head()

In [ ]:
#Compute two new columns as the difference between max and min
sppExtent['XRange'] = sppExtent.maxX - sppExtent.minX
sppExtent['YRange'] = sppExtent.maxY - sppExtent.minY
sppExtent.head()